In [5]:
import pandas as pd
import numpy as np
import matplotlib as mp
import math as math
import matplotlib.pyplot as plt

# Question 1

In [11]:
def buffon(disc_diameters):
    num_tosses=4444444
    line_spacing=1

    results = {}
    
    # Loop through each disc diameter
    for diameter in disc_diameters:
        radius = diameter / 2
        line_crossings = {}

        # keep track of 5 lines
        #cross_counts = np.zeros(10)
        cross_counts = {i: 0 for i in range(10)}
        
        # Run Monte Carlo simulation
        for _ in range(num_tosses):
            # randomly place the center of the disk based on the line space size
            center_y = np.random.random() * line_spacing
            
            # dtermine how far the disk reaches 
            lower_extent = center_y - radius
            upper_extent = center_y + radius
            
            # count to keep track of how many lines are crossed
            lines_crossed = 0
            
            # check lower line
            if lower_extent <= 0:
                lines_crossed += 1
                
            # check top ine
            if upper_extent >= line_spacing:
                lines_crossed += 1
                
            # check any other lines above 
            line_pos = 2 * line_spacing
            while upper_extent >= line_pos:
                lines_crossed += 1
                line_pos += line_spacing
                
            # check lines below
            line_pos = -line_spacing
            while lower_extent <= line_pos:
                lines_crossed += 1
                line_pos -= line_spacing
                
            # increment count in dict for the cross lines count 
            cross_counts[lines_crossed] += 1
            
        # and then calculate the probabilities
        # for i in range(len(cross_counts)):
        #     if cross_counts[i] > 0:
        #         line_crossings[i] = cross_counts[i] / num_tosses

        # for key, value in cross_counts.items():
        #     if(value == 0):
        #         cross_counts.pop(key)
        #         continue
        #     cross_counts[key] = cross_counts[key] / num_tosses

        for key in list(cross_counts.keys()):
            if cross_counts[key] == 0:
                cross_counts.pop(key)
            else:
                cross_counts[key] /= num_tosses
                
        results[diameter] = cross_counts
        
    return results

In [12]:
disc_diameters = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.5, 2.0, 3.0]

results = buffon(disc_diameters)

In [13]:
for d, dic in results.items():
    print("---------------------------------------------------")
    print(f"d = {d}")
    for line_cross_count, prob in dic.items():
        if prob != 0:
            print(f"P({line_cross_count} lines) = {prob}")

---------------------------------------------------
d = 0.1
P(0 lines) = 0.899948789994879
P(1 lines) = 0.100051210005121
---------------------------------------------------
d = 0.2
P(0 lines) = 0.7998154549815455
P(1 lines) = 0.2001845450184545
---------------------------------------------------
d = 0.3
P(0 lines) = 0.7000193950019395
P(1 lines) = 0.2999806049980605
---------------------------------------------------
d = 0.4
P(0 lines) = 0.6000847350084735
P(1 lines) = 0.3999152649915265
---------------------------------------------------
d = 0.5
P(0 lines) = 0.499759249975925
P(1 lines) = 0.500240750024075
---------------------------------------------------
d = 0.6
P(0 lines) = 0.400092340009234
P(1 lines) = 0.599907659990766
---------------------------------------------------
d = 0.7
P(0 lines) = 0.2996579549657955
P(1 lines) = 0.7003420450342045
---------------------------------------------------
d = 0.8
P(0 lines) = 0.1999572949957295
P(1 lines) = 0.8000427050042705
--------------

# MAKE PLOT FOR THIS QUESTION

# Question 2: